[![Open In Colab](./colab-badge.png)](https://colab.research.google.com/github/MooseNeuro/moose-notebooks/blob/main/Action_potentials_K_channel.ipynb) [![Binder](./binder_logo.png)](https://mybinder.org/v2/gh/MooseNeuro/moose-notebooks/HEAD?labpath=Action_potentials_K_channel.ipynb)

# Synaptic connection between two Hodgkin-Huxley type neurons
In this notebook we will model synaptic connection in a more realistic scenario. We will create two single-compartmental neurons `A` and `B` with Hodgkin and Huxley's K+ and Na+ channel dynamics.

Then we will connect `A` to `B` by a synapse. Here `A` is the presynaptic compartment and `B` is the postsynaptic compartment. Note that although we are not differentiating between soma, axon, and dendrites here, in a morphologically detailed model you will designate some compartments as presynaptic terminals and others as postsynaptic terminals, and create synapses between these.

When the synapse is activated, the synaptic conductance will have a fast rising phase and a slow falling phase. This is modeled as a double-exponential with two time constants $\tau_{1}$ and $\tau_{2}$. See the [Synapses](Synapses.ipynb) notebook for a simpler version called $\alpha$-synapse.

In [ ]:
## Only required on colab! Uncomment to enable.
# !pip install pymoose --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import moose

First we shall create two identical models of the squid giant axon (as we did in the [Action potentials notebook](Action_potentials.ipynb).

For complex models, it is more convenient to create a library of reusable prototypes and then copy them for the actual model. The `/library` element is created for this.

## Setup single-compartment Hodgkin-Huxley neurons
First we define a function to create a prototype of a single-compartment with Hodgkin-Huxley's Na+ and K+ channels.

We have switched to modern convention of considering the outside medium as ground, setting intracellular membrane potential to -70 mV (Hodgkin and Huxley set their ground to the intracellular medium).

We have also modified the equations to switch from physiological units (ms, mV) to SI units.

The `get_hh_proto()` function creates a prototype compartment under `/library` only if it does not exist already. Otherwise it simply returns the existing compartment.

In [ ]:
EREST_ACT = -70e-3
ENa = 115e-3 + EREST_ACT
EK = -12e-3 + EREST_ACT
DIA = 30e-6

In [ ]:
def get_hh_proto():
    lib = moose.Neutral('/library')
    comp_path = f'{lib.path}/comp'
    if moose.exists(comp_path):
        return moose.element(comp_path)
        
    # Make a prototype compartment
    proto_comp = moose.Compartment(comp_path)
    sarea = np.pi * DIA * DIA  # Surface area of spherical compartment
    proto_comp.Em = EREST_ACT + 10.613e-3    # Hodgkin and Huxley used resting voltage as 0
    proto_comp.initVm = EREST_ACT
    proto_comp.Cm = 1e-2 * sarea  # 1 uF/cm^2 = 1e-2 F/m^2
    proto_comp.Rm = 1 / (10 * 0.3 * sarea) # Gm = 0.3 mS/cm^2 = 0.3 * 10 S/m^2
    
    # Create the HH channels
    na_chan = moose.HHChannel(f'{proto_comp.path}/Na')
    k_chan = moose.HHChannel(f'{proto_comp.path}/K')
    moose.connect(na_chan, 'channel', proto_comp, 'channel')
    moose.connect(k_chan, 'channel', proto_comp, 'channel')
    na_chan.Ek = ENa
    na_chan.Gbar = 10 * 120 * sarea  # 1 mS/cm^2 = 10 S/m^2
    k_chan.Ek = EK
    k_chan.Gbar = 10 * 36 * sarea  # 1 mS/cm^2 = 10 S/m^2
    # Set the gate powers in order to instantiate the gate elements
    na_chan.Xpower = 3   # m^3
    na_chan.Ypower = 1   # h
    
    m_gate = moose.element(na_chan.gateX)
    h_gate = moose.element(na_chan.gateY)
    
    k_chan.Xpower = 4
    n_gate = moose.element(k_chan.gateX) 
    
    # Note: we are converting physiological units in the original 
    # HH formulae to SI units. Also, we are considering resting membrane potential
    # EREST_ACT to be -70 mV while Hodgkin and Huxley considered it to be 0 mV.
    # Therefore we must shift the voltage by EREST_ACT to use HH-formulae.
    # We are converting V into mV before passing it to the original HH equations.
    # The rates are then multiplied by 1e3 to convert from 1/ms to 1/s.
    m_gate.alphaExpr = f'1e3 * 0.1 * (25 - 1e3 * (v - ({EREST_ACT})))/(exp((25 - 1e3 * (v - ({EREST_ACT})))/10) - 1)'
    m_gate.betaExpr =  f'1e3 * 4 * exp(- 1e3 * (v - ({EREST_ACT}))/ 18)'
    h_gate.alphaExpr = f'1e3 * 0.07 * exp(- 1e3 * (v - ({EREST_ACT}))/ 20)'
    h_gate.betaExpr = f'1e3 / (exp((30 - 1e3 * (v - ({EREST_ACT}))) / 10) + 1)'
    n_gate.alphaExpr = f'1e3 * 0.01 * (10 - 1e3 * (v - ({EREST_ACT}))) / (exp((10 - 1e3 * (v - ({EREST_ACT})))/10) - 1)'
    n_gate.betaExpr = f'1e3 * 0.125 * exp(-1e3 * (v - ({EREST_ACT})) / 80)'
    
    # Set the interpolation table ranges and number of divisions for the gates
    # and fill the tables by evaluating the expressions
    for gate in (m_gate, h_gate, n_gate):  # avoid repetition
        gate.min = -110e-3
        gate.max = 50e-3
        gate.divs = 1000
        gate.useInterpolation = True
        gate.fillFromExpr()   # fill the tables by evaluating alpha/beta expressions
    return proto_comp

Now make containers for model and data, and make two copies of the prototype for the two neurons.

In [ ]:
model = moose.Neutral('/model')
data = moose.Neutral('/data')

In [ ]:
# Make copies of the prototype compartment
proto = get_hh_proto()
nrn_a = moose.copy(proto, model, 'A')
nrn_b = moose.copy(proto, model, 'B')

Below we print some debug information to 

In [ ]:
moose.le(model)
na = moose.element(f'{proto.path}/Na')
print('Total GNa:', na.Gbar)
m = moose.element(f'{na.path}/gateX')
print('alpha_m expression:', m.alphaExpr)
print('vmin:', m.min, 'vmax:', m.max, 'vdivs:', m.divs)
print('m-tableA computed:', m.tableA)
print('m-tableB computed:', m.tableB)
h = moose.element(f'{na.path}/gateY')
print('h-tableA computed:', h.tableA)
print('h-tableB computed:', h.tableB)

## Create and setup the synapse
In this model we will create a synapse from neuron `A` to neuron `B`.

To detect threshold crossing on the presynaptic compartment `A`, and send the spike event time to the postsynaptic partner, we need a `SpikeGen` object as part of the presynaptic mechanism. The `Compartment` sends out its voltage from the `src` field `VmOut` and the `SpikeGen` receives this value into its `dest` field `Vm` .

On the postsynaptic compartment we need a synaptic channel that undergoes conductance change due to spike events. This is modeled by the `SynChan` class. The `SynChan` element is reciprocally connected to the postsynaptic `Compartment` via the `channel` field, just like `HHChannel`. 

For receiving the spike events and passing it to the `SynChan`, a `SynHandler` element is needed. The `SynHandler` accumulates incoming spike information and sends it out through the `src` field `activationOut`, and this goes into the `dest` field `activation` on the `SynChan`. 

In [ ]:
spikegen = moose.SpikeGen(f'{nrn_a.path}/spike')
# SpikeGen receives compartment's voltage from `VmOut` source field into its `Vm` dest field
moose.connect(nrn_a, 'VmOut', spikegen, 'Vm')  

synchan = moose.SynChan(f'{nrn_b.path}/synchan')
moose.connect(synchan, 'channel', nrn_b, 'channel')

synhandler = moose.SimpleSynHandler(f'{synchan.path}/synh')
moose.connect(synhandler, 'activationOut', synchan, 'activation')

A `SynHandler` can take incoming spikes from multiple sources (presynaptic terminals). We must specify the number of synapses in the `numSynapses` field. This will allocate one element to the `synapse` array (of class `Synapse`) on the `SynHandler` to receive spike events.

In [ ]:
synhandler.numSynapses = 1

 Now we can connect the `SpikeGen` to the corresponding element of the `synapse` array on the `SynHandler`. The `src` field `spikeOut` of the `SpikeGen` will be connected to the `dest` field `addSpike` of the element on the `Synapse`.

In [ ]:
moose.connect(spikegen, 'spikeOut', synhandler.synapse[0], 'addSpike')

We now set the synaptic weight to `1.0` and delay to `1 ms`.

In [ ]:
synhandler.synapse[0].weight = 1.0
synhandler.synapse[0].delay = 1e-3   # 1 ms synaptic delay

We also set the properties of the `SynChan`. Like Hodgkin-Huxley type ion channels, synaptic channels also have a reversal potential, set through the field `Ek`. As the value here is positive compared to the resting membrane potential of the postsynaptic compartment, the channel will have an depolarizing or excitatory effect when activated.

`Gbar` is the maximal conductance. We assign different values to the two time constants, `tau1` and `tau2`, giving it a double-exponential dynamics. The double-exponential synapse has a conductance


$g(t) = A\ g_{max}\ \frac{e^{-t/\tau_{1}} - e^{-t/\tau_{2}}}{\tau_{1} - \tau_{2}}$

Here $A$ is a normalization constant to ensure that $g(t)$ reaches the maximum value of $g_{max}$.

In [ ]:
# Ek is the reversal potential.
# Making it positive relative to the resting membrane potential
synchan.Ek = 0.0
synchan.Gbar = 1e-7
synchan.tau1 = 1e-3  # 1 ms time constant
synchan.tau2 = 5e-3

## Setup current clamp to stimulate presynaptic neuron
We also create a current clamp to artificially stimulate the presynaptic compartment in order to make it spike.

In [ ]:
cclamp = moose.PulseGen(f'{nrn_a.path}/input')
moose.connect(cclamp, 'output', nrn_a, 'injectMsg')
cclamp.firstDelay = 50e-3
cclamp.firstWidth = 200e-3
cclamp.firstLevel = 1e-9  

## Setup data recording
Finally, we need to setup the `Table`s for data recording. We track the voltage of neurons `A` and `B` with two `Tables`: `vm_a` and `vm_b`. We monitor the conductance (`Gk` field) of the `SynChan` with `gsyn`.

In [ ]:
vm_a = moose.Table(f'{data.path}/vm_a')
moose.connect(vm_a, 'requestOut', nrn_a, 'getVm')
vm_b = moose.Table(f'{data.path}/vm_b')
moose.connect(vm_b, 'requestOut', nrn_b, 'getVm')

gsyn = moose.Table(f'{data.path}/gsyn')
moose.connect(gsyn, 'requestOut', synchan, 'getGk')

## Run simulation

In [ ]:
simtime = 300e-3
moose.reinit()
moose.start(simtime)

## Plot data

Now we can plot the `Vm` of neurons `A` and `B` along with the synaptic conductance.

In [ ]:
fig, axes = plt.subplots(nrows=3, sharex='all')
t = np.arange(len(gsyn.vector)) * gsyn.dt * 1e3
axes[0].plot(t, vm_a.vector)
axes[0].set_ylabel('Vm (Volt)')
axes[0].set_title('Neuron A')
axes[1].plot(t, vm_b.vector)
axes[1].set_ylabel('Vm (Volt)')
axes[1].set_title('Neuron B')
axes[2].plot(t, gsyn.vector * 1e6)
axes[2].set_xlabel('Time (ms)')
axes[2].set_xlabel('Gk (uS)')
axes[2].set_title('Gk of SynChan on B')
fig.tight_layout()

# Exercises

1. What do you observe in the membrane potential plot of Neuron B? If you want to pass on the output of B to another identical neuron C, would the same `threshold` work for the `SpikeGen` on B?
2. The maximal conductance of the synapse `Gbar` was set to $0.1 \mu S$. Is that really the maximum value in the simulation?
4. Modify the weight of of the synapse and note the results.
5. Modify `Gbar` of the `SynChan` and note the results.
6. Increase `Ek` of the `SynChan` and note the effect.
7. Change `Ek` of the `SynChan` to $-80 mV$ and observe the result.
8. How will you model a GABA-ergic (inhibitory synapse)?